In [1]:
%load_ext autoreload
%load_ext line_profiler

In [2]:
%autoreload 1

In [3]:
import sys

# Allow importing game modules.
sys.path.append('../')

In [4]:
# Auto reload wordle libraries on edit -> save
%aimport wordle
%aimport solver

In [5]:
%aimport wordle_game.helper

In [6]:
import numpy as np

from tqdm.notebook import tqdm

In [7]:
wordlist = wordle.WordList()

In [12]:
guesses = []

for target in tqdm(wordlist.answers):
    for guess in wordlist.acceptable_guesses:
        guesses.append((target, guess, wordle_game.helper.generate_state(target, guess)))

  0%|          | 0/2315 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
%timeit wordle.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

18.1 µs ± 909 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
%timeit wordle_game.helper.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

4.22 µs ± 221 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
%lprun -f wordle.generate_state wordle.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

In [14]:
%lprun -f wordle_game.helper.generate_state wordle_game.helper.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

C:\Users\ahaqu\Anaconda3\lib\site-packages\line_profiler\line_profiler.py:307: UserWarning: Could not extract a code object for the object <built-in function generate_state>
  profile = LineProfiler(*funcs)
